Summary
-------

This note illustrates an easily overlooked but significant overhead of `scipy.linalg`'s Cholesky solver, [`scipy.linalg.cho_solve`](https://scipy.github.io/devdocs/generated/scipy.linalg.cho_solve.html#scipy.linalg.cho_solve).

Motivation
----------

The Cholesky solver solves the linear system $A x = b$ for Hermitian positive-definite matrix $A$ using its Cholesky factorization, which is calculated by [`scipy.linalg.cho_factor`](https://scipy.github.io/devdocs/generated/scipy.linalg.cho_factor.html#scipy.linalg.cho_factor).  The functions are wrappers for LAPACK functions `?POTRS` and `?POTRF` respectively, and their performance largely depends on the underyling LAPACK implementation.

A nice feature of `scipy` is of course its safety.  Both functions include an optional check for infinities and NaNs in its input arrays.  This check is on by default.

However, the cost associated with the check can be pretty high.  Currently, this check is done by the `numpy` function `numpy.isfinite` which does *not* short-circuit on first non-finite (or first NaN) element!  The function returns a whole Boolean array with as many elements as its input.  Then, in the implementation of the function `numpy.asarray_chkfinite`, the whole Boolean array is `AND`ed (short-circuitedly, using `numpy.ndarray.all()` method) to see if there's a `False`.  This adds an $\mathcal{O}(n^2)$ overhead.

The following benchmark, with commentaries, shows how large this overhead can be.

Benchmark
---------

First, we mock up some numbers for benchmarking.

In [1]:
import numpy
import scipy.linalg


N = 2048
a = numpy.random.random(N)
b = numpy.random.random(N)

We need some Hermitian, positive-definite matrix as input.  This can be done using the outer-product trick.

In [2]:
c = numpy.outer(a, a)
c[numpy.diag_indices_from(c)] += 5.0 + numpy.random.random(N)

This matrix `c` is, by construction, Hermitian and positive-definite.  If you're paranoid about its symmetry, you can always do this:

In [3]:
c = c + c.T
c *= 0.5

Now the Cholesky factor of `c`:

In [4]:
cfac = scipy.linalg.cho_factor(c)

We're ready for the benchmark now.

In [5]:
t_chk = %timeit -o scipy.linalg.cho_solve(cfac, b)

100 loops, best of 3: 11.9 ms per loop


In [6]:
t_nochk = %timeit -o scipy.linalg.cho_solve(cfac, b, check_finite=False)

100 loops, best of 3: 3.3 ms per loop


The speed-up is...

In [7]:
print "Speed-up (best-of-%d): %.1f%%" % (t_chk.repeat, (t_chk.best / t_nochk.best - 1) * 100)

Speed-up (best-of-3): 259.3%


Wow!

---

Just to make sure in this case we're not missing anything by turning off `check_finite`:

In [8]:
numpy.allclose(scipy.linalg.cho_solve(cfac, b), scipy.linalg.cho_solve(cfac, b, check_finite=False))

True

Finally, let's see if there are any CPU cycles to squeeze by calling the LAPACK function directly (without the other administrative overhead of `scipy`):

In [9]:
%timeit scipy.linalg.lapack.dpotrs(cfac[0], b, lower=cfac[1], overwrite_b=False)

100 loops, best of 3: 3.29 ms per loop


It turns out there's not much to squeeze.  In fact, it may be worse sometimes.  (This may be a limitation of running `%timeit` in a notebook.  Actually, if it's run directly in `IPython`, there's indeed a tiny, but positive speed-up.)

Conclusion
----------

There's a massive overhead with `check_finite=True` in the Cholesky linear-system solver.  The check protects against possible non-termination of the LAPACK function, but when you're *sure* there's no such risk, removing the check could boost performance by a lot.  Really a lot.